# Importing the Necessary Modules

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from wordcloud import STOPWORDS, WordCloud
import requests
import re
import time
import pickle

# Declaring Variables For Crawling

In [ ]:
base_addr1 = 'https://www.bankbazaar.com/reviews/'
base_addr2 = '/all-products.html?reviewPageNumber='
instit_name = []
reviews = []
ratings = []
extra_info = []
product = []
banks = []

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:81.0) Gecko/20100101 Firefox/81.0'}

**Assigning Institutions**

In [ ]:
instit_name = ['ing-vysya-bank','allahabad-bank','idfc-bank','idbi-bank','central-bank-of-india']

# Defining Crawler

In [ ]:
def pagefinder(head_s,search_name):
  headers = head_s
  url = base_addr1+search_name+base_addr2+'1'
  #print (url)
  r = requests.get(url, headers=headers)
  soup = BeautifulSoup(r.text, 'html.parser')
  containers = soup.find_all(attrs={'class':'pagination-review'})
  page_s = containers[0].find_all('span')[0].text
  l1 = page_s.split()
  page_count = int(l1[3])
  return page_count

In [ ]:
def Spider(search_name,page_count):
  for N in range(1,page_count):
    url = base_addr1 + search_name + base_addr2 + str(N)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    containers = soup.find_all(name='li',attrs={'class':'review-box'})
    for container in containers:
      product.append(container.find('div',attrs={'class':'review-bank-title'}).text)
      reviews.append(container.find(name='div', attrs={'class':'text_here review-desc-more'}).text)
      ratings.append(container.find_all('span')[4].text)
      extra_info.append(container.find('div',attrs={'class':'reviewer-profile'}).text)
      banks.append(search_name)
    time.sleep(4)

# Building the Reviews Dataset

In [ ]:
for bank_name in instit_name:
  page_count = pagefinder(headers,bank_name)
  Spider(bank_name,page_count)

In [ ]:
for val in product:
  product.remove(val)
  val = val.strip('\n')
  #print(val)
  product.append(val)

In [ ]:
for val in reviews:
  reviews.remove(val)
  val = val.strip('\n')
  #print(val)
  reviews.append(val)

In [ ]:
bank_review = pd.DataFrame(list(zip(banks,product,ratings,reviews)), columns =['Bank', 'Service','Rating','Review'])

In [ ]:
bank_review.head()

,Bank,Service,Rating,Review
0,ing-vysya-bank,\n\nOther Loan,3.0,\n Earlier I ha...
1,ing-vysya-bank,\n\nOther Loan,3.0,\n I've a savin...
2,ing-vysya-bank,\n\nOther Loan,4.0,\n Accessibilit...
3,ing-vysya-bank,\n\nOther Loan,3.0,\n I hold a sav...
4,ing-vysya-bank,\n\nOther Loan,4.0,\n I had the fi...


In [ ]:
bank_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2580 entries, 0 to 2579
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Bank     2580 non-null   object 
 1   Service  2580 non-null   object 
 2   Rating   2580 non-null   float64
 3   Review   2580 non-null   object 
dtypes: float64(1), object(3)
memory usage: 80.8+ KB


In [ ]:
bank_review['Rating'] = bank_review['Rating'].astype('float')

In [ ]:
cleaned_Service = []
for service in bank_review.Service:
  if service[0:1]== '\n' :
    service = service[2:]
  if service[0:1]== '\n' :
    service = service[2: ]
  cleaned_Service.append(service)

bank_review['Service'] = cleaned_Service

In [ ]:
bank_review['Review'] = bank_review['Review'].str.strip()

In [ ]:
review_result = []
for val in bank_review.Rating:
  if val == 5.0:
    review_result.append(2)
  elif val >= 4.0 and val < 5.0 :
    review_result.append(1)
  else:
    review_result.append(0)

bank_review['Result'] = review_result

In [ ]:
with open('banks','wb') as f:
  pickle.dump(banks,f)
with open('product','wb') as f:
  pickle.dump(product,f)
with open('ratings','wb') as f:
  pickle.dump(ratings,f)
with open('reviews','wb') as f:
  pickle.dump(reviews,f)

In [ ]:
bank_review.to_csv('bank_review.csv',encoding= 'UTF-8')

In [ ]:
from google.colab import files
files.download('bank_review.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ls 

sample_data
